# Run fine-tuning on MoleculeNet

In [1]:
%cd ..

/home/adam/Projects/hybrid-transformer


In [1]:
import os
import argparse
import json

import wandb
from tqdm import tqdm
from hybrid_transformer.configs.task import TaskConfig
from hybrid_transformer.configs.model import ModelConfig
from hybrid_transformer.configs.trainer import TrainerConfig
from hybrid_transformer.configs.logger import LoggerConfig

from hybrid_transformer.utils.datasets.auto import AutoDataset
from hybrid_transformer.utils.tokenizers.auto import AutoTokenizer
from hybrid_transformer.models.auto import AutoModel
from hybrid_transformer.utils.loggers.wandb import WandbLogger

from hybrid_transformer.trainers.trainer import Trainer

from scripts.joint_learning.train import DEFAULT_CONFIG_FILES

from hybrid_transformer.utils.objectives.guacamol.objective import GUACAMOL_TASKS
from hybrid_transformer.models.prediction import PREDICTION_MODEL_CONFIGS
from hybrid_transformer.utils.objectives.molecule_net.objective import MOLECULE_NET_REGRESSION_TASKS


from scripts.pretrain.eval import DEFAULT_REFERENCE_FILE, evaluate_distribution_learning

%load_ext autoreload
%autoreload 2

/home/adam/miniconda3/envs/hybrid-transformer/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-04 18:09:38.115854: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-04 18:09:38.117617: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-04 18:09:38.141349: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-04 18:09:38.141364: E external/local_xla/xla/stream_executor/cu

In [6]:
DEFAULT_CONFIG_FILES = {
    'trainer': "./configs/trainers/finetune-prediction/",
    'logger': "./configs/loggers/wandb/"
}

task_config_path = lambda: f'./configs/tasks/molecule_net/{task}/config.json'
TASKS = MOLECULE_NET_REGRESSION_TASKS
try:
    PREDICTION_MODEL_CONFIGS.pop('GPTForPrediction')
    PREDICTION_MODEL_CONFIGS.pop('JointGPT')
except:
    pass

for task in tqdm(TASKS):
    task_config = TaskConfig.from_pretrained(task_config_path())
    task_config.augmentation_prob = 0.0  # disable augmentation
    dataset = AutoDataset.from_config(task_config, split='test')
    tokenizer = AutoTokenizer.from_config(task_config)

    for model_name, path_to_model_config in tqdm(PREDICTION_MODEL_CONFIGS.items()):

        model_config = ModelConfig.from_pretrained(path_to_model_config)

        trainer_config = TrainerConfig.from_pretrained(DEFAULT_CONFIG_FILES['trainer'])
        logger_config = LoggerConfig.from_pretrained(DEFAULT_CONFIG_FILES['logger'])
        logger_config.wandb_log = False

        run_dir = f'{model_name}/{task}'
        out_dir = os.path.join('./notebooks', run_dir)
        trainer_config.out_dir = out_dir
        
        logger_config.project = 'Table 1 ' + 'bug'
        logger_config.name = model_name + '_' + task

        model = AutoModel.from_config(model_config)
        logger = WandbLogger(logger_config, [task_config, model_config, trainer_config])
        trainer = Trainer(config=trainer_config, model=model, tokenizer=tokenizer, logger=logger)
        
        results_prediction = trainer.test(dataset)






  0%|          | 0/2 [00:00<?, ?it/s]

number of parameters: 38.06M
tokens per iteration will be: 61,440
Using cuda device
Random seed set to 1337
num decayed parameter tensors: 63, with 38,115,840 parameters
num non-decayed parameter tensors: 25, with 12,800 parameters
using fused AdamW: True
Compiling model..


  0%|          | 0/3 [00:09<?, ?it/s]


BackendCompilerFailed: backend='inductor' raised:
BrokenProcessPool: A child process terminated abruptly, the process pool is not usable anymore

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


You can suppress this exception and fall back to eager by setting:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True
